# HW5 : Math for Robotics

Author: Ruffin White  
Course: CSE291  
Date: Mar 23 2018

In [ ]:
# Make plots inline
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'svg')
set_matplotlib_formats('png', 'pdf')
# set_matplotlib_formats('png')

# import modules for plotting and data analysis
import matplotlib.pyplot as plt

## 1. 

> The world model is shown in figure 1. The robot is a differential drive
system of size 50x50.

1. Generate the configuration space for the robot with a grid size of 2x2 and 5 deg in angular resolution. Generate an illustration of what the configuration space looks like with the robot at orientations 0, 45 and 90 deg.
2. Use greedy search find the shortest path between start-point (50,50) and end-point (750,250). Illustrate the path and provide its length.
3. Compute the safest past from start to finish (hint: medial axis transform). Illustrate the path and provide its length.
4. Use probabilistic roadmaps (PRM) to compute a path between startand end-points with 50, 100 and 500 sample points. What is the difference in path length? Illustrate each computed path.
5. Do the same with Rapid exploring random trees (RRT). What are the main differences in performance between PRM and RRT? Illustrate each path.

In [ ]:
import cv2
import numpy as np
import pandas as pd
import pathlib
import scipy.spatial
import scipy.signal

from collections import OrderedDict
from PIL import Image

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
scale = 2

width = int(800/scale)
hight = int(300/scale)
seg1 = int(200/scale)
seg2 = int(600/scale)
seg3 = int(400/scale)
seg4 = int(100/scale)

map_2d = np.zeros(shape=(width, hight))
map_2d[0,:] = 1
map_2d[:,0] = 1
map_2d[:,hight-1] = 1
map_2d[width-1,:] = 1

map_2d[seg1-1,0:seg1] = 1
map_2d[seg2-1,0:seg1] = 1

map_2d[seg3-1,hight-1-seg1:hight-1] = 1
map_2d[seg3-1-seg4:seg3-1+seg4,hight-1-seg1] = 1

In [ ]:
rwidth = int(50/scale)
rhight = int(50/scale)
robot_2d = np.ones(shape=(rwidth, rhight))

In [ ]:
f, axs = plt.subplots(1,1,figsize=(24,64))
axs.imshow(map_2d.T, cmap=plt.cm.hot_r, interpolation='none')
for d in ["left", "top", "bottom", "right"]:
    axs.spines[d].set_visible(False)
plt.show()

In [ ]:
points = np.asarray(np.where(map_2d == 1)).T
vor = scipy.spatial.Voronoi(points)

In [ ]:
vertices = vor.vertices[::1].T.astype(int)
map_2d_vert = map_2d.copy()
map_2d_vert[vertices[0],vertices[1]] = 2

In [ ]:
f, axs = plt.subplots(1,1,figsize=(24,64))
axs.imshow(map_2d_vert.T, cmap=plt.cm.hot_r, interpolation='none')
for d in ["left", "top", "bottom", "right"]:
    axs.spines[d].set_visible(False)
plt.show()

In [ ]:
grad = scipy.signal.convolve2d(map_2d, robot_2d, boundary='symm', mode='same')
# grad = np.clip(a=grad, a_min=0, a_max=1)

In [ ]:
f, axs = plt.subplots(1,1,figsize=(24,64))
axs.imshow(grad.T, cmap=plt.cm.Spectral_r, interpolation='none')
for d in ["left", "top", "bottom", "right"]:
    axs.spines[d].set_visible(False)
plt.show()

In [ ]:
def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

In [ ]:
robot_2d_rot = rotate_bound(robot_2d, 70)

In [ ]:
grad = scipy.signal.convolve2d(map_2d, robot_2d_rot, boundary='symm', mode='same')
# grad =  (grad > 0).astype(int)

In [ ]:
f, axs = plt.subplots(1,1,figsize=(24,64))
axs.imshow(grad.T, cmap=plt.cm.Spectral_r, interpolation='none')
for d in ["left", "top", "bottom", "right"]:
    axs.spines[d].set_visible(False)
plt.show()